# Общие функции и константы для всех блоков модели

In [1]:
import sqlalchemy as sa
import warnings
from os import path, getcwd
import datetime as dt
from scipy.stats import rv_continuous
from scipy.integrate import quad

warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import os
import sys
module_path = os.path.abspath(os.path.join('..', 'PY'))

if module_path not in sys.path:
     sys.path.append(module_path)
        
import pandas as pd
from source_data.src import DBSource, RowTypes
import source_data.prepare as prep

Класс для определения namespace

In [3]:
path.join('..','DB', 'year.sqlite3')

'..\\DB\\year.sqlite3'

In [89]:
class common:
    
    strYearDBPath=path.join('..','DB', 'year.sqlite3') # путь к рабочей базе данных фактических рядов SQLite 
    strExYearDBPath=path.join('..','DB', 'exog_year.sqlite3') # путь к рабочей базе данных экзогенных переменных
    strExParamDBPath=path.join('..','DB', 'exog_param.sqlite3') # путь к рабочей базе данных экзогенных параметров, задаваемых вручную
    strSvodDBPath=path.join('..','DB', 'svod.sqlite3') # путь к рабочей базе данных итоговых рядов SQLite 

    # константы базы данных SQLite3
    strDatas_table='datas' # название таблицы с данными в базе данных SQLite
    strHeaders_table='headers' # название таблицы с текстовыми и кодовыми именами рядов в базе данных SQLite
    
    def make_select_year_string(rowCodes, datas_table=strDatas_table, header_table=strHeaders_table):
        assert type(rowCodes) in (str, list, tuple), 'make_select_year_string from() COMMON module: error in param type'
        strSelect='''select {datas_table}.date, {datas_table}.value, {header_table}.code2 from {datas_table} join {header_table}
                      on {datas_table}.code={header_table}.code {where_condiiton}'''


        if type(rowCodes)==str:
            return strSelect.format(where_condiiton = 'where {header_table}.code2=="{code2}"'.format(header_table=header_table, code2=rowCodes),
                                   datas_table=datas_table, header_table=header_table)
        elif type(rowCodes) in (tuple, list):
            return strSelect.format(where_condiiton = 'where {header_table}.code2 in {codes2}'.format(header_table=header_table, 
                                                                                                      codes2=tuple(rowCodes)),
                                   datas_table=datas_table, header_table=header_table)
        return None

    def make_frame(pdf):
        k=pdf.set_index(['date', 'code2']).unstack().reset_index().set_index('date')
        k.columns=[c[1] for c in k.columns]
        return k

    def scale(pdf, list_fields=[], multiplier=1):
        pdf[list_fields] *= multiplier
        return pdf

    def add(pdf, list_fields=[], param=0):
        pdf[list_fields] += param
        return pdf
    
    def combine_frames(*lst_source_data:DBSource):
        def _combine(pdf, dct, new_frame_key):
            try:
                return pdf.combine_first(dct[new_frame_key].make_frame())
            except KeyError:
                return pdf

            
        """
        правильно соединяет все источники данных для моделей
        основной фрейм - база данных, если в нем что-либо равно np.nan то
           для таких точек значения берутся из фрейма экзогенных рядов. Если после этого в результате есть np.nan то
             для таких точек значения берутся из фрейма экзогенных параметров. Если после этого в результате есть np.nan то
               для таких точек значения берутся из фрейма модельных расчетов
        если какой-либо фрейм остуствует, он, понятное дело, не комбинируется 
        Параметры
        : переменные класса source_data.src.DBSource (не меньше одного, не больше 4)
        
        Выхлоп:
        : pandas DataFrame, собранный и готовый
        """
        lst_keys=[i.row_type.name for i in lst_source_data]
        
        if len(lst_keys) != len(set(lst_keys)):
            _e=[(v.row_type.name, i) for i, v in enumerate(lst_source_data)]
            raise KeyError('Несколько входных переменный имеют одинаковый row_type: {}'.format(_e))
            
        _dct={i.row_type.name:i for i in lst_source_data}
        _pdf=pd.DataFrame(None)
        _pdf=_combine(_pdf, _dct, 'FACT')
        _pdf=_combine(_pdf, _dct, 'EXOG_R')
        _pdf=_combine(_pdf, _dct, 'EXOG_P')
        _pdf=_combine(_pdf, _dct, 'MODEL')
        
        return _pdf
    
    def update_dt_s(datetime_value=dt.datetime.now()):
        return datetime_value.strftime('%Y-%m-%d %H:%M:%S %f')

    def update_dt_d(string_value=None):
        assert string_value is not None, 'на входе нужно строкое представление datetime в формате "%Y-%m-%d %H:%M:%S %f"'
        s=string_value.split(' ')
        l_dt=s[0].split('-')
        l_tm=s[1].split(':')
        i_msec=int(s[2])
        return dt.datetime(year=int(l_dt[0]), month=int(l_dt[1]), day=int(l_dt[2]), 
                           hour=int(l_tm[0]), minute=int(l_tm[1]), second=int(l_tm[2]), microsecond=int(i_msec))
    
    class kolmakov_gen(rv_continuous):
        """ Распределения доходов населения по уровню среднедушевого среднемесячного денежного дохода в денежном выражении """
        
        def _pdf(self, x, sigma, mu):
            Px = np.exp(-(np.log(x)-mu)**2 / (2*sigma**2))
            return Px / (sigma*np.sqrt(2*np.pi))
    #         return lognorm.pdf(x, s=sigma, scale=np.exp(mu)) * x

        def _cdf(self, x, sigma, mu):
            return quad(self._pdf, 0, x, args=(sigma, mu))[0] 


    kolmakov = kolmakov_gen(a=0.0, name='kolmakov', shapes='sigma, mu')


## Ниже - тесты функций. Чтобы работало, нужно убрать магию

In [91]:
%%script false --no-raise-error

lstYearCodes=['squareddy_ss_x', 'CPIAv', 'Pop_x', 'HHAv', 'Unmpl_s', 'DispInc_rI', 'Inc_x', 'p_ProbDef', 
              'UZUss_x', 'MEPop', 'DispPop', 'LivMin', 'p_superrich', 'sdelkiddy_ss_x', 'sdelkikp_ss_x', 
              'squarekp_ss_x', 'DispInc_I','oldpravaddy_x','longarenda','izavarijnogo',
              'kapremont','partinZSK','Badzhilfond_x','sdelkikp_x'] # список экзогенных и фактических данных
 

lstSvod=['price1mddy_alt_x', 'price1mall_alt_x', 'AvSqDdy', 'AvSqVtor','VvodyIZDunits','VvodyMKD',
     'VvodyMKD_inst','VvodyMKD_gov','VvodyIZD','VvodyMKDunits','sdelkikp_x','sdelkiddy_x'] # ре

srcAct = DBSource(common.strYearDBPath, RowTypes.FACT, lstYearCodes)
srcAct.prepare=[{'func':prep.scale, 'list_fields':['Inc_x',], 'param':1e3},
                {'func':prep.scale, 'list_fields':['sdelkiddy_ss_x',], 'param':1e6}]

srcExog = DBSource(common.strExYearDBPath, RowTypes.EXOG_R, lstYearCodes)

srcExogP = DBSource(common.strExParamDBPath, RowTypes.EXOG_P, lstYearCodes)

srcSvod  = DBSource(common.strSvodDBPath, RowTypes.MODEL, lstSvod)
srcSvod.prepare={'func':prep.scale, 'list_fields':['price1mddy_alt_x', 'price1mall_alt_x'], 'param':1e3}


pdfWork=(srcAct.make_frame().combine_first(srcExog.make_frame())
                   .combine_first(srcExogP.make_frame())
                   .combine_first(srcSvod.make_frame()))


common.combine_frames(srcAct, srcExog, srcExogP, srcSvod)


,AvSqDdy,AvSqVtor,Badzhilfond_x,CPIAv,DispInc_I,DispPop,HHAv,Inc_x,LivMin,MEPop,...,p_superrich,partinZSK,price1mall_alt_x,price1mddy_alt_x,sdelkiddy_ss_x,sdelkiddy_x,sdelkikp_ss_x,sdelkikp_x,squareddy_ss_x,squarekp_ss_x
date,,,,,,,,,,,,,,,,,,,,,
2004,NaN,NaN,NaN,1.108812,1.104000,NaN,2.677054,6410.300000,2376.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,65.703978,NaN,NaN,1.127055,1.124000,0.756100,2.661470,8088.300000,3018.0,8.711611,...,0.02,NaN,21048.567265,18491.692819,1.439358e+11,NaN,1.334440,1.443503,NaN,NaN
2006,65.107090,NaN,NaN,1.096830,1.135000,0.769907,2.645886,10154.800000,3422.0,8.928732,...,0.02,NaN,28597.165335,24937.896936,1.529254e+11,NaN,1.417784,1.564560,NaN,NaN
2007,64.216907,NaN,NaN,1.089881,1.121000,0.785663,2.630302,12540.200000,3847.0,9.129498,...,0.02,NaN,37938.493305,32765.902784,1.624764e+11,NaN,1.506332,1.703862,NaN,NaN
2008,63.243902,NaN,2.333110,1.141048,1.024000,0.782878,2.614717,14863.600000,4593.0,9.301331,...,0.02,NaN,44576.706072,37788.915681,5.073128e+11,NaN,1.265722,1.550739,NaN,65.564414
2009,61.675214,50.877922,2.910700,1.116977,1.030000,0.783533,2.599133,16895.000000,5153.0,9.427587,...,0.02,NaN,42565.501231,36473.861415,2.826073e+11,NaN,1.182876,1.299313,NaN,60.182264
2010,60.086697,50.985166,2.584900,1.068508,1.059000,0.782280,2.583549,18958.400000,5688.0,9.546797,...,0.02,NaN,41870.584649,35853.805771,1.175073e+11,0.145883,1.655528,1.925781,7.060626,84.407359
2011,58.132834,51.084428,2.387000,1.084634,1.005000,0.775441,2.544156,20780.000000,6369.0,9.644999,...,0.02,NaN,44815.617500,37897.472700,1.614052e+11,0.233760,1.983726,2.427822,9.382939,101.337510
2012,56.425400,51.185577,2.043990,1.050667,1.046000,0.781314,2.504763,23221.100000,6510.0,9.747765,...,0.02,NaN,50629.950501,41865.338092,2.047766e+11,0.331445,2.074330,2.623857,11.554604,106.175779
